# Setting up equations using algorithmic differentiation 

*This tutorial is deprecated. It will be updated during an ongoing overhaul of tutorials.*

This tutorial is meant as an introduction to the PorePy framework for defining and working with (non-linear) equations, using the mixed-dimensional flow problem as an example. For an introduction to basic concepts of algorithmic differentiation, please refer to the dedicated [automatic differentiation tutorial](https://github.com/pmgbergen/porepy/blob/develop/tutorials/automatic_differentiation.ipynb). 
Note that for several much used problems, PorePy provides `Model` classes to solve mixed-dimensional PDEs. These classes use the AD functionality as described herein. A less technical explanation of how to solve the present mixed-dimensional problem is provided in the [incompressible flow tutorial](https://github.com/pmgbergen/porepy/blob/develop/tutorials/incompressible_flow_model.ipynb), which also details the equations. Herein, we provide more detailed explanations and descriptions of the design of the AD objects.

## Framework components
The framework consists of four types of classes:
1. Variables. These carry the numerical state of the primary variables, and also values at previous time steps and iteration states.
2. Grid-dependent operators are defined on one or multiple subdomain grids. Examples are:
    * divergence and trace operators
    * boundary conditions
    * parameter arrays and matrices
    * projections between interfaces and subdomain grids
    * projections between sets of subdomains and subsets.
3. Discretization objects. These are essentially shells around standard PorePy discretization methods. Their purpose is to provide access to the discretization matrices computed by those methods in a form consistent with the AD framework.
4. Classes for book-keeping, notably degree-of-freedom handling and the relation between equations/variables and the underlying mixed-dimensional grid.

Solving (partial differential) equations on a mixed-dimensional grid requires defining the above objects, combining them into equations and solving the ensuing equation system. 

## Test case: A mixed-dimensional grid.
As a test case, we define a mixed-dimensional grid, which we for simplicity let be Cartesian

In [4]:
import numpy as np
import porepy as pp


import scipy.sparse.linalg as spla

# fractures 1 and 2 form a T-intersection in (3, 3)
frac_1 = np.array([[2, 2], [2, 4]])
frac_2 = np.array([[2, 5], [3, 3]])
# fracture 3 is isolated
frac_3 = np.array([[6, 6], [1, 5]])

mdg = pp.meshing.cart_grid([frac_1, frac_2, frac_3], nx=np.array([7, 7]))

Next, we define the PDE's variables and parameters on the subdomains and interfaces.

In [5]:
# String representations of the variables.
pressure_var = 'pressure'
mortar_var = 'mortar_flux'
param_key = 'flow'

# Loop over all subdomains
for sd, data in mdg.subdomains(return_data=True):
    # Define a cell centered variable
    data[pp.PRIMARY_VARIABLES] = {pressure_var: {'cells': 1}}
    # Assign an initial numerical value
    pp.set_solution_values(
        name=pressure_var, 
        values=np.random.rand(sd.num_cells), 
        data=data, 
        time_step_index=0)
    # Set default parameters for the flow problem
    pp.initialize_default_data(sd, data, param_key, {})
    
# Also loop over interfaces
for intf, data in mdg.interfaces(return_data=True):
    data[pp.PRIMARY_VARIABLES] = {mortar_var: {'cells': 1}}
    pp.set_solution_values(
        name=mortar_var, 
        values=np.random.rand(intf.num_cells), 
        data=data, 
        time_step_index=0)
    kn = 1e-1 * np.ones(intf.num_cells)
    pp.initialize_data(intf, data, param_key, {'normal_diffusivity': kn})


### Mixed-dimensional AD variables and book-keeping
The next step is to define Ad representations of the (mixed-dimensional) variables. For this, we first need to define a degree of freedom manager (`DofManager`) and an equation manager (`EquationManager`). The `DofManager` is responsible for keeping track of the degrees of freedom in the mixed-dimensional system, whereas the `EquationManager` is responsible for providing the Ad  representations of the variables.

*NOTE*: A consistent ordering of subdomains is crucial, as it sets the ordering of variables, discretization objects, etc. Thus, the same ordering should be used throughout the simulation.

In [6]:
dof_manager = pp.DofManager(mdg)
equation_manager = pp.ad.EquationManager(mdg, dof_manager)
p = equation_manager.merge_variables([(sd, pressure_var) for sd in mdg.subdomains()])
lmbda = equation_manager.merge_variables([(intf, mortar_var) for intf in mdg.interfaces()])

Note that `p` and `lmbda` do not have numerical values. What we have done instead is:
1. Prepare the ground to write the abstract equations, and
2. Prepare for the subsequent translation of the equations to numerical representation (values and derivatives)

### Grid-related operators
Now, we are ready to apply define AD objects for this mixed-dimensional problem. The key to exploit this efficiently (in terms of both userfriendliness and computational speed) is to operate on several subdomains simultaneously. For instance, the mass conservation equation requires a divergence operator, which we define jointly for all subdomains. We also require a representation of boundary conditions.

In [7]:
div = pp.ad.Divergence(subdomains=mdg.subdomains())
# The boundary condition object is initialized with the parameter key used to set the parameters in the
# above loop. This is critical to ensure parameters are not mixed in multi-physics simulations.
bound_ad = pp.ad.BoundaryCondition(param_key, subdomains=mdg.subdomains())

AttributeError: module 'porepy.numerics.ad' has no attribute 'BoundaryCondition'

Note that these are not matrices, but a special object:

In [ ]:
type(div)

porepy.numerics.ad.grid_operators.Divergence

We will come back to how to translate `div` into a numerical expression. Similarly, `bound_ad` is not a numerical boundary condition, but rather a way to access given boundary data.

We can also define merged projection operators between subdomain grids and mortar grids. This can be done either on the whole `mdg` or on parts of it. The ordering of the grids matters- you most likely will not get consistent results if the ordering is altered throughout the simulation (if you get a warning, disregard it; this will be handled at a later point):

In [ ]:
mortar_proj = pp.ad.MortarProjections(mdg=mdg, subdomains=mdg.subdomains(), interfaces=mdg.interfaces())

c:\users\ivar\onedrive - university of bergen\documents\github\porepy\src\porepy\grids\mortar_grid.py:801: UserWarning: Is it really meaningful to ask for signs of a one sided mortar grid?
  warnings.warn(


### Discretization objects
Next, we turn to discretization. To be compatible with the Ad framework, PorePy discretizations need a wrapper which mainly allows for the delayed evaluation of the expressions. For instance, the Ad version of Mpfa is defined by writing

In [ ]:
mpfa = pp.ad.MpfaAd(param_key, mdg.subdomains())

This object, once again, has no numerical values but is rather an abstract representation of a standard Mpfa discretization. The two versions of Mpfa refer to the discretization matrices resulting from the discretization in similar ways: Mpfa has attributes like `flux_matrix_key`, which specifies where the flux discretization matrix is stored inside the `data` dictionary. Similarly, MpfaAd has an attribute `flux`, which, upon parsing of an Ad expression (below), will access the same discretization matrix.

Finally, we define a discretization object for the interface equation

In [ ]:
robin = pp.ad.RobinCouplingAd(param_key, mdg.interfaces())

## Mixed-dimensional ad equations
To explore how the mixed-dimensional Ad framework works in action, we can define the flux discretization on all subdomains as

In [ ]:
interior_flux = mpfa.flux @ p

In essence, there are two types of Ad objects:
1. Atomic objects, like `mpfa.flux` and `p`. These can be considered pointers to places in the data dictionary where the numerical values associated with the objects are stored. For instance, `p` in our example points to a collection of `data[pp.STATE][pressure_var]`, where `data` is the data dictionary for each of the subdomains on which `p` is defined.
2. Composite objects, like `interior_flux`, formed by combining Ad objects (which themselves can be atomic or composites) using basic mathematical operations. 

These Ad objects are not designed for numerical evaluation by themselves, they can be thought of as recipes for combining discretizations, variables, etc. To parse a recipe, we provide it with a `MixedDimensionalGrid`, from where it can pull numerical values for variables, discretization matrices, and so on.

In [ ]:
interior_flux.discretize(mdg)
num_flux = interior_flux.evaluate(dof_manager)
print(num_flux)

UnboundLocalError: local variable 'data' referenced before assignment

We note that, since the `mpfa` object was defined on all subdomains, `num_flux` has the size of the total number of faces in the grid. Its Jacobian matrix is a mapping from all the declared primary variables in the full `MixedDimensionalGrid` (in this case, pressure in the cells on subdomains and fluxes for all interface cells) to the faces on all subdomains.


We can define more elaborate combinations of variables. The `interior_flux` object represents only the part of the flux caused by pressure variations in the interior of the subdomains. To get the _full flux_, we need to account for boundary conditions from external boundaries, as well as from internal boundaries to neighboring subdomains of lower dimensions.

In [ ]:
full_flux = (
    interior_flux 
    + mpfa.bound_flux @ (bound_ad 
                         + mortar_proj.mortar_to_primary_int @ lmbda
                        )
    )

In the lower-dimensional subdomains, the projected interface fluxes manifest as sources.
Put together, we now have the full mass conservation equation on all subdomains:

In [ ]:
conservation = div @ full_flux + mortar_proj.mortar_to_secondary_int @ lmbda

We can also define equations for the interfaces. To that end, we first define the pressure trace on internal boundaries - the most accurate representation of this trace is a bit complex within Mpfa (and Tpfa) methods

In [ ]:
pressure_trace_from_primary = (
    mortar_proj.primary_to_mortar_avg @ mpfa.bound_pressure_cell @ p
    + mortar_proj.primary_to_mortar_avg * mpfa.bound_pressure_face * mortar_proj.mortar_to_primary_int * lmbda
    )


Now, we can write the Darcy-type equation for the interface flux

In [ ]:
interface_flux_eq = (
    robin.mortar_discr * (pressure_trace_from_primary
                          - mortar_proj.secondary_to_mortar_avg * p)
    + lmbda)

### Assemble the system of equations
Now, we only have to feed the equations to the equation manager to be ready to assemble the full system, formed by the conservation statement and the interface flux equations:

In [ ]:
eqs = {'subdomain_conservation': conservation, 'interface_fluxes': interface_flux_eq}
equation_manager.equations.update(eqs)

The `equation_manager` can be used to assemble the coupled linear system, much in the same way as a recipe is evaluated. Before that, the discretization matrices must be constructed.

**NOTE**: The computed solution has the interpretation of the update to the existing state, that is, the random values we assigned above. The solution must be distributed in an additive manner. 

In [ ]:
# first discretize
equation_manager.discretize(mdg)
# next assemble the equations
A, b = equation_manager.assemble()

# Solve system, note the minus sign on the right hand side
solution = spla.spsolve(A, b)

# Distribute variable to local data dictionaries
dof_manager.distribute_variable(solution, additive=True)

exporter = pp.Exporter(mdg, 'ad_test')
exporter.write_vtu([pressure_var])



## What have we done
We summarize the steps needed to define an equation:
1. Define variables 
2. Define grid-related operators (not strictly necessary, but most often)
3. Define discretizations
4. Combine into equations, and evaluate.